In [ ]:
#Load Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from random import randrange, uniform
#from fancyimpute import KNN

In [ ]:
# Working directory 

In [ ]:
os.chdir("D:\\Data Science-edwisor\\Case Study\\Market_WorkingSpace\\")
os.getcwd()

In [ ]:
# Load the data
marketing_train = pd.read_csv("marketing_train.csv")

In [ ]:
marketing_train.shape      # display the no. of obs and variable

In [ ]:
marketing_train.head()     # display only top 5 records

In [ ]:
# there is one target variable which tells us that cust will respond to the capmaign or not

In [ ]:
# Exploratory data analysis, clean the data and replace with some meaningful data
marketing_train['schooling'] = marketing_train['schooling'].replace("illiterate", "unknown")
marketing_train['schooling'] = marketing_train['schooling'].replace(["basic.4y", "basic.6y","basic.9y", "high.school", "professional.course"], "high.school")
marketing_train['default'] = marketing_train['default'].replace("yes", "unknown")
marketing_train['marital'] = marketing_train['marital'].replace("unknown", "married")
marketing_train['month'] = marketing_train['month'].replace(["sep","oct","mar","dec"], "dec")
marketing_train['month'] = marketing_train['month'].replace(["aug","jul","jun","may","nov"], "jun")
marketing_train['loan'] = marketing_train['loan'].replace("unknown", "no")
marketing_train['profession'] = marketing_train['profession'].replace(["management","unknown","unemployed","admin."], "admin.")
marketing_train['profession'] = marketing_train['profession'].replace(["blue-collar","housemaid","services","self-employed","entrepreneur","technician"], "blue-collar")

# Missing Value Analysis

In [ ]:
# create a dataframe for missing value analysis
missing_val = pd.DataFrame(marketing_train.isnull().sum())

In [ ]:
missing_val

In [ ]:
# reset index 
missing_val = missing_val.reset_index()
missing_val

In [ ]:
# Rename column name
missing_val = missing_val.rename(columns = {"index": "Variables" , 0:"Missing_Percentage"})
missing_val

In [ ]:
# Calculate the percentage
missing_val['Missing_Percentage'] = (missing_val['Missing_Percentage']/len(marketing_train))*100
missing_val

In [ ]:
# sort the data in the proper order and save in to output file
missing_val = missing_val.sort_values('Missing_Percentage', ascending=False).reset_index(drop = True)

# Save output
missing_val.to_csv("Misisng_perc.csv", index = False)

In [ ]:
# Handle missing values and lets create missing value on our own
marketing_train['custAge'].loc[70]

In [ ]:
#imputaion method
#Actual value  = 29
#mean = 40.01
#meadin = 38
#KNN = 42.12

#so, we can freeze  median

In [ ]:
marketing_train['custAge'].loc[70] = np.nan

In [ ]:
marketing_train['custAge'].loc[70]

In [ ]:
# impite with mean and median
#marketing_train['custAge'] = marketing_train['custAge'].fillna(marketing_train['custAge'].mean())

marketing_train['custAge'] = marketing_train['custAge'].fillna(marketing_train['custAge'].median())
marketing_train['custAge'].loc[70]

In [ ]:
# KNN imputation 
# Convert all the categorical into level(numnerical) becuase dist can only be cal with num 

lis = []
for i in range(0, marketing_train.shape[1]):       # iterate over all columns(0-row, 1- column)
    if(marketing_train.iloc[:,i].dtypes == 'object'):
        marketing_train.iloc[:,i] = pd.Categorical(marketing_train.iloc[:,i])
        marketing_train.iloc[:,i] = marketing_train.iloc[:,i].cat.codes
        marketing_train.iloc[:,i] = marketing_train.iloc[:,i].astype('object')
        
        lis.append(marketing_train.columns[i])

In [ ]:
#Apply KNN
#marketing_train = pd.DataFrame(KNN(k=3).fit_transform(marketing_train), columns = marketing_train.columns)

In [ ]:
#Convert into proper datatypes
for i in lis:
  #  marketing_train.loc[:,i] = marketing_train.loc[:,i].round()
    marketing_train.loc[:,i] = marketing_train.loc[:,i].astype('object')

# Outliers Analysis

In [ ]:
marketing_train.head()

In [ ]:
df = marketing_train.copy()

In [ ]:
%matplotlib inline   

plt.boxplot(marketing_train['custAge'])

In [ ]:
# Lets put continous variable in cnmaes beacuse outliers only apply in continous variable
cnames = ['custAge', 'campaign','pdays','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed','pmonths','pastEmail']

In [ ]:
cnames

In [ ]:
#calculation of box plot
for i in cnames:
    print(i)
    q75, q25 = np.percentile(marketing_train.loc[:,i], [75,25])
    iqr = q75 - q25
    
    min = q25 - (iqr*1.5)
    max = q75 + (iqr*1.5)
    
    print(min)
    print(max)
    
    marketing_train = marketing_train.drop(marketing_train[marketing_train.loc[:,i] < min].index)
    marketing_train = marketing_train.drop(marketing_train[marketing_train.loc[:,i] > max].index)
    

In [ ]:
marketing_train.shape

In [ ]:
# in the second approach, we can replace outliers with NA and then handle missing value analysis to it

# Feature selection

In [ ]:
#apply correlation analysis and can be apply only on continous variable
df_corr = marketing_train.loc[:,cnames]

In [ ]:
#set the height and width of the plot
f, ax = plt.subplots(figsize = (7,5))

# generate the coorelation matrix
corr = df_corr.corr()

#Plot using seaborn plot
sns.heatmap(corr, mask = np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square = True, ax=ax)

In [ ]:
# chi2 test for categorical variable
cat_names = ['profession', 'marital', 'schooling', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome',
            ]

In [ ]:
#loop for chi2 test
for i in cat_names:
    print(i)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(marketing_train['responded'], marketing_train[i]))
    print(p)
    
# it will return 4 values ie chi squer, p value, degree of freedom and expected value
# if p value <0.05 then null hypothesis else reject

In [ ]:
# remove unnecesary varibale 
marketing_train = marketing_train.drop(['pdays', 'emp.var.rate','day_of_week', 'loan', 'housing'], axis = 1)

In [ ]:
marketing_train.shape

# Feature Scaling

In [ ]:
#df = marketing_train.copy()
marketing_train = df.copy()

In [ ]:
#Normility check - only in continoous variable  just to check whter the distribution is normaliy distributed or not
%matplotlib inline
plt.hist(marketing_train['custAge'], bins = 'auto')

In [ ]:
marketing_train.shape

In [ ]:
cnames = ['custAge', 'campaign','previous','cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'pmonths', 'pastEmail']
cnames

In [ ]:
#Normalization method
#for i in cnames:
#    print(i)
#    marketing_train[i] = (marketing_train[i] - min(marketing_train[i]))/(max(marketing_train[i]) - min(marketing_train[i]))

In [ ]:
marketing_train.describe()

In [ ]:
#Standarization
for i in cnames:
    print(i)
    marketing_train[i] = (marketing_train[i] - marketing_train[i].mean())/marketing_train[i].std()

In [ ]:
marketing_train.isnull().sum()

# Sampling

In [ ]:
#Simple random sampling
sim_sample = marketing_train.sample(3000)
sim_sample

In [ ]:
sim_sample = sim_sample.drop(['previous','pmonths', 'pastEmail'], axis = 1)

In [ ]:
sim_sample

In [ ]:
marketing_train = marketing_train.drop(['previous','pmonths', 'pastEmail'], axis = 1)

In [ ]:
marketing_train

In [ ]:
#Systematic random sampling
#Generate K number and then sample
k = len(marketing_train)//(3500)

In [ ]:
#Random number generate using random simple sampling
RandNum = randrange(0,5)
RandNum

In [ ]:
#select the Kth observations starting from randnum 
sys_sample = marketing_train.iloc[RandNum::k, :]
sys_sample

In [ ]:
#Stratified sampling
from sklearn.model_selection import train_test_split

In [ ]:
#select categorical variable
y = marketing_train['profession']

#select subset using startified sampling
Rest, Sample = train_test_split(marketing_train, test_size = 0.6, stratify = y)

In [ ]:
Sample

# Model Development

In [ ]:
#Import libraries for model development for decision tree
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
#assigning target categories with Yes and No
#Decision Tree classifier for clasiification
#Beacuse decsion tree want target variable to be categorical
marketing_train['responded'] = marketing_train['responded'].replace(0, 'No')
marketing_train['responded'] = marketing_train['responded'].replace(1, 'Yes')

In [ ]:
marketing_train

In [ ]:
#divide the data into train and test 
X = marketing_train.values[:,0:16]
Y = marketing_train.values[:,-1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
#decision tree algo -- C5.0 - entropy and cart - gini
C50_model = tree.DecisionTreeClassifier(criterion = 'entropy').fit(X_train, Y_train)

In [ ]:
#predict new test case
y_pred = C50_model.predict(X_test)

In [ ]:
#create dot file to visulize tree   #http://webgraphviz.com
dotfile = open('pt.dot', 'w')
df = tree.export_graphviz(C50_model, out_file=dotfile, feature_names = marketing_train.columns[0:16])

In [ ]:
#decision tree for regression
#for dependent variable to be continous so we use regression..in the next notebook beacuse here dependent is categorical

# Confusion matrix

In [ ]:
#build confusion matrix
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(Y_test, y_pred)

In [ ]:
CM

In [ ]:
CM = pd.crosstab(Y_test, y_pred)

In [ ]:
CM

In [ ]:
#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FP = CM.iloc[0,1]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]

In [ ]:
#Check the accuracy of the model
accuracy_score(Y_test, y_pred)*100

In [ ]:
#Accuracy
((TN + TP)*100)/(TP + TN + FN + FP)

In [ ]:
#We can calculate False neagtive rate, false positive rate, recall, specificity ect from this confusion matrix

In [ ]:
#False negative rate
(FN * 100)/(FN + TP)             # 69% is very high and we need to reduce it

In [ ]:
#Recall
(TP*100)/(TP + FN)             #30%

In [ ]:
#Results from model
#Accuracy - 82%
#FNR - 68%

# Random Forest Algorith

In [ ]:
#import random forest library from sklean and dev model
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF_model = RandomForestClassifier(n_estimators = 100).fit(X_train, Y_train)

In [ ]:
RF_prediction = RF_model.predict(X_test)

In [ ]:
RF_prediction

In [ ]:
#let build confusion matrix to check our model accuracy
CM = pd.crosstab(Y_test, RF_prediction)

TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1] 

#accuracy
((TP+TN)*100)/(TP + TN + FP+ FN)

#FNR
#(FN*100)/(FN + TP)

In [ ]:
#Results from this 
#Accuracy = 88394%
#FNR = 72.62%
# so freeze n=100 and this model

# Logistic Regression

In [ ]:
#let prepare data for logistic regression and used for categorical variable
#replace target variable with Yes and No
marketing_train['responded'] = marketing_train['responded'].replace('No',0)
marketing_train['responded'] = marketing_train['responded'].replace('Yes',1)

In [ ]:
#save logistic data in proper dataframe
marketing_train_logit = pd.DataFrame(marketing_train['responded'])

In [ ]:
marketing_train_logit

In [ ]:
#Add continous variable to this dataframe
marketing_train_logit = marketing_train_logit.join(marketing_train[cnames])

In [ ]:
marketing_train_logit

In [ ]:
#create dummies for categorical variables
cat_names = ["profession", "marital", "schooling", "default", "contact", "month", "poutcome"]

In [ ]:
for i in cat_names:
    temp = pd.get_dummies(marketing_train[i], prefix = i)
    marketing_train_logit = marketing_train_logit.join(temp)

In [ ]:
Sample_Index = np.random.rand(len(marketing_train_logit)) < 0.8

train = marketing_train_logit[Sample_Index]
test = marketing_train_logit[~Sample_Index]

In [ ]:
#select column indexes for independent variable
train_cols = train.columns[1:31]

In [ ]:
train_cols

In [ ]:
#Build logistic regression
import statsmodels.api as sm
logit = sm.Logit(train['responded'], train[train_cols]).fit()
logit.summary()

In [ ]:
#Predict test data",
test['Actual_prob'] = logit.predict(test[train_cols])
test['ActualVal'] = 1
test.loc[test.Actual_prob < 0.5, 'ActualVal'] = 0

In [ ]:
test.head()

In [ ]:
#let build confusion matrix to check our model accuracy
CM = pd.crosstab(test['responded'], test['ActualVal'])

TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1] 

#accuracy
((TP+TN)*100)/(TP + TN + FP+ FN)

#FNR
#(FN*100)/(FN + TP)

In [ ]:
#Accuracy = 89.90%
#FNR = 71.91

# KNN implemetation

In [ ]:
#KNN imputation and implementaion for model
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors = 9).fit(X_train, Y_train)

In [ ]:
#predict test case
KNN_predictions = KNN_model.predict(X_test)

In [ ]:
#let build confusion matrix to check our model accuracy
CM = pd.crosstab(Y_test, KNN_predictions)

TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1] 

#accuracy
((TP+TN)*100)/(TP + TN + FP+ FN)

#FNR
#(FN*100)/(FN + TP)

In [ ]:
#Accuracy = 88%
#FNR = 82.14%

# Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
NB_model = GaussianNB().fit(X_train, Y_train)

In [ ]:
#predict test cases
NB_predictions = NB_model.predict(X_test)

In [ ]:
NB_predictions

In [ ]:
#let build confusion matrix to check our model accuracy
CM = pd.crosstab(Y_test, NB_predictions)

TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1] 

#accuracy
((TP+TN)*100)/(TP + TN + FP+ FN)

#FNR
#(FN*100)/(FN + TP)

In [ ]:
#Accuracy = 83% approx
#FNR = 54.76
#So, model seems good with this condition

In [ ]:
# this above all algorithm is for supervised learning and we have finished this case study
#For unsupervised learnign diff dataset is used